## Estadística Aplicada G44
Actividad 6 - 25 Sep

Cinthia Zayde Gabrielle Garcia Belmonte 2076329

In [78]:
# Librerias
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [110]:
# Librerias especializadas 
from lifelines import CoxPHFitter, ExponentialFitter, WeibullFitter, LogNormalFitter
import seaborn as sns

Cada vez que un empleado renuncia a una empresa “X”, a esta empresa le toma como 
mínimo 6 meses contratar un remplazo, más 3 meses de capacitación y un mes de 
adaptación. Tal situación implica pérdida de tiempo y recursos por los siguientes 
motivos:

a) Cuando un empleado se va, sus responsabilidades deben ser redistribuidas entre los empleados restantes. Esto puede enerar una carga de trabajo adicional para aquellos que quedan, lo que puede llevar a la fatiga 

b) Reemplazar a un empleado puede ser costoso. Incluye gastos en reclutamiento, selección, capacitación y adaptación

c) La rotación frecuente de empleados puede llevar a interrupciones en el flujo de trabajo, ya que los nuevos empleados ecesitan tiempo para adaptarse y alcanzar el mismo nivel de productividad que sus predecesores


## Base de datos

**renunciar:**  1:renunció, 0:censurado

**semestre:** Semestres que ha trabajado en la empresa

**satisfacción:** Que tan satisfecho se encuentra la persona con su trabajo en una escala de 0 a 10

**Evaluación:** Que tan bien realiza su trabajo la persona en una escala de 0 a 10

**proyectos:** Número de proyectos asignados al empleado

**promedio_horas_mes:** Promedio de horas mensuales trabajadas

**accidentes:** 1: ha tenido un accidente dentro de la empresa, 0: no ha tenido accidentes dentro de la empresa

**ascenso_5years:** 1: si el empleado ha sido ascendido durante los últimos 5 años, 0: si no ha sido ascendido durante los ultimos 5 años

**departamento:** Nombre del departamento en el que trabaja 

**salario:** Clasificación del salario en alto, medio y bajo

In [80]:
desercion_laboral = pd.read_csv('https://raw.githubusercontent.com/jimmyzac/Estadistica-Aplicada-FCFM-UANL/main/bases_datos/desercion_laboral.csv')

In [81]:
desercion_laboral.head()

,satisfaccion,evaluacion,proyectos,promedio_horas_mes,semestre,accidente,renunciar,ascenso_5years,departamento,salario
0,3.8,5.3,2,157,3,0,1,0,ventas,bajo
1,8.0,8.6,5,262,6,0,1,0,ventas,medio
2,1.1,8.8,7,272,4,0,1,0,ventas,medio
3,7.2,8.7,5,223,5,0,1,0,ventas,bajo
4,3.7,5.2,2,159,3,0,1,0,ventas,bajo


### a) Verificar que no haya missing values, en caso de haberlos decidir imputar o eliminar

In [82]:
# Verificar missing values
desercion_laboral.isnull().sum()

satisfaccion           0
evaluacion             0
proyectos              0
promedio_horas_mes     0
semestre               0
accidente              0
renunciar              0
ascenso_5years         0
departamento          26
salario                0
dtype: int64

In [83]:
# la col departamento tiene 26 valores perdidos, hay que eliminarlos
desercion_laboral = desercion_laboral.dropna()

In [84]:
# Verificar que se eliminaran missing values
desercion_laboral.isnull().sum()

satisfaccion          0
evaluacion            0
proyectos             0
promedio_horas_mes    0
semestre              0
accidente             0
renunciar             0
ascenso_5years        0
departamento          0
salario               0
dtype: int64

### b) Verificar que no haya valores duplicados con dataframe.duplicated().sum() en caso de haber valores duplicados eliminarlos con dataframe=dataframe.drop_duplicates(keep='first').reset_index(drop=True)

In [85]:
# Verificar valores duplicados
desercion_laboral.duplicated().sum()

2982

In [86]:
# Si hay valores duplicados, hay que eliminarlos
desercion_laboral = desercion_laboral.drop_duplicates(keep = 'first').reset_index(drop = 'True')

In [87]:
desercion_laboral.duplicated().sum()

0

### c) Verificar que todas las variables sean numéricas, si no son numéricas hacerlas numéricas, en el caso de las categóricas convertirlas en dummies y conservar n-1 dummies


In [88]:
# Tipo de variables
desercion_laboral.dtypes

satisfaccion          float64
evaluacion            float64
proyectos               int64
promedio_horas_mes      int64
semestre                int64
accidente               int64
renunciar               int64
ascenso_5years          int64
departamento           object
salario                object
dtype: object

In [89]:
# Convertir los flotantes a enteros
desercion_laboral['satisfaccion'] = desercion_laboral['satisfaccion'].astype('int64')
desercion_laboral['evaluacion'] = desercion_laboral['evaluacion'].astype('int64')

In [90]:
#Dummies de la columna departamento
dummies_departamento = pd.get_dummies(desercion_laboral['departamento'], prefix='departamento').astype('int64')
dummies_departamento.head()



,departamento_Investigación,departamento_administración,departamento_contabilidad,departamento_desarrollo de producto,departamento_informática,departamento_mantenimiento,departamento_mercadotecnia,departamento_recursos humanos,departamento_soporte,departamento_ventas
0,0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,1


In [91]:
## departamento_desarrollo de producto eliminada
dummies_departamento = dummies_departamento[['departamento_Investigación', 'departamento_administración', 'departamento_contabilidad', 'departamento_informática', 'departamento_mantenimiento', 'departamento_mercadotecnia', 'departamento_recursos humanos', 'departamento_soporte', 'departamento_ventas']]

In [92]:
# Dummies de la columna Salario
dummies_salario = pd.get_dummies(desercion_laboral['salario'], prefix='salario').astype('int64')
dummies_salario.head()

,salario_alto,salario_bajo,salario_medio
0,0,1,0
1,0,0,1
2,0,0,1
3,0,1,0
4,0,1,0


In [93]:
dummies_salario = dummies_salario[['salario_alto', 'salario_medio']]

In [94]:
desercion_laboral = pd.concat([desercion_laboral, dummies_departamento, dummies_salario], axis=1)
desercion_laboral = desercion_laboral.drop('departamento', axis=1)
desercion_laboral = desercion_laboral.drop('salario', axis=1)

In [95]:
# Tipo de variables
desercion_laboral.dtypes

satisfaccion                     int64
evaluacion                       int64
proyectos                        int64
promedio_horas_mes               int64
semestre                         int64
accidente                        int64
renunciar                        int64
ascenso_5years                   int64
departamento_Investigación       int64
departamento_administración      int64
departamento_contabilidad        int64
departamento_informática         int64
departamento_mantenimiento       int64
departamento_mercadotecnia       int64
departamento_recursos humanos    int64
departamento_soporte             int64
departamento_ventas              int64
salario_alto                     int64
salario_medio                    int64
dtype: object

### d) Calcular estadísticas descriptivas

In [96]:
# estadisticas descriptivas
desercion_laboral.describe()

,satisfaccion,evaluacion,proyectos,promedio_horas_mes,semestre,accidente,renunciar,ascenso_5years,departamento_Investigación,departamento_administración,departamento_contabilidad,departamento_informática,departamento_mantenimiento,departamento_mercadotecnia,departamento_recursos humanos,departamento_soporte,departamento_ventas,salario_alto,salario_medio
count,11991.000000,11991.000000,11991.000000,11991.000000,11991.000000,11991.000000,11991.000000,11991.000000,11991.000000,11991.000000,11991.000000,11991.000000,11991.00000,11991.000000,11991.000000,11991.000000,11991.000000,11991.000000,11991.000000
mean,5.851889,6.707948,3.802852,200.473522,3.364857,0.154282,0.166041,0.016929,0.057877,0.036361,0.051789,0.081394,0.18714,0.056125,0.050121,0.151864,0.270119,0.082562,0.438746
std,2.397506,1.703189,1.163238,48.727813,1.330240,0.361234,0.372133,0.129012,0.233520,0.187194,0.221610,0.273451,0.39004,0.230173,0.218204,0.358904,0.444040,0.275230,0.496254
min,0.000000,3.000000,2.000000,96.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,4.000000,5.000000,3.000000,157.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,6.000000,7.000000,4.000000,200.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,8.000000,8.000000,5.000000,243.000000,4.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000
max,10.000000,10.000000,7.000000,310.000000,10.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


### 4. Estimar el modelo de Cox e interpretar cada uno de los coeficientes

In [97]:
## Estimasción del modelo
cph = CoxPHFitter().fit(desercion_laboral,'semestre','renunciar')
cph.print_summary()

<lifelines.CoxPHFitter: fitted with 11991 total observations, 10000 right-censored observations>
             duration col = 'semestre'
                event col = 'renunciar'
      baseline estimation = breslow
   number of observations = 11991
number of events observed = 1991
   partial log-likelihood = -15905.76
         time fit was run = 2023-09-26 03:07:34 UTC

---
                                coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                                             
satisfaccion                   -0.24       0.79       0.01            -0.25            -0.22                 0.78                 0.80
evaluacion                     -0.01       0.99       0.01            -0.04             0.01                 0.96                 1.01
proyectos                      -0.28       0.75       0.02            -0.32            -0.24                 0.73                 0.78
promedio_horas_mes              0.00       1.00       0.00             0.00             0.00                 1.00                 1.00
accidente                      -1.24       0.29       0.10            -1.44            -1.05                 0.24                 0.35
ascenso_5years                 -1.37       0.25       0.36            -2.07            -0.67                 0.13                 0.51
departamento_Investigación     -0.35       0.70       0.14            -0.64            -0.07                 0.53                 0.93
departamento_administración    -0.13       0.88       0.17            -0.46             0.21                 0.63                 1.23
departamento_contabilidad      -0.09       0.91       0.14            -0.36             0.17                 0.70                 1.19
departamento_informática       -0.05       0.95       0.12            -0.29             0.19                 0.74                 1.21
departamento_mantenimiento      0.09       1.09       0.11            -0.12             0.30                 0.88                 1.35
departamento_mercadotecnia     -0.01       0.99       0.13            -0.27             0.26                 0.76                 1.29
departamento_recursos humanos   0.10       1.10       0.13            -0.16             0.36                 0.85                 1.44
departamento_soporte            0.11       1.11       0.11            -0.11             0.32                 0.89                 1.38
departamento_ventas             0.01       1.01       0.10            -0.20             0.21                 0.82                 1.24
salario_alto                   -1.54       0.21       0.15            -1.83            -1.25                 0.16                 0.29
salario_medio                  -0.39       0.68       0.05            -0.48            -0.30                 0.62                 0.74

                                cmp to      z      p   -log2(p)
covariate                                                      
satisfaccion                      0.00 -26.22 <0.005     500.95
evaluacion                        0.00  -0.97   0.33       1.60
proyectos                         0.00 -14.30 <0.005     151.75
promedio_horas_mes                0.00   4.10 <0.005      14.55
accidente                         0.00 -12.39 <0.005     114.67
ascenso_5years                    0.00  -3.85 <0.005      13.08
departamento_Investigación        0.00  -2.43   0.01       6.06
departamento_administración       0.00  -0.74   0.46       1.13
departamento_contabilidad         0.00  -0.68   0.50       1.01
departamento_informática          0.00  -0.41   0.68       0.56
departamento_mantenimiento        0.00   0.81   0.42       1.26
departamento_mercadotecnia        0.00  -0.05   0.96       0.05
departamento_recursos humanos     0.00   0.74   0.46       1.12
departamento_soporte              0.00   0.95   0.34       1.55
departamento_ventas               0.

**evaluacion, departamento_administración, departamento_contabilidad, departamento_informática, departamento_mantenimiento, departamento_mercadotecnia, departamento_recursos humanos, departamento_soporte, departamento_ventas** Su P-valor es mayor que el $\alpha = 0.05$, por lo que NO se rechaza $H_0$, lo que quiere decir que estas variables no tiene efecto en el riesgo de que un empleado renuncie a la empresa

**satisfaccion** El P-Valor es menor que $\alpha = 0.05$, entonces Se Rechaza $H_0$. Mientras más satisfecho se encuentre el empleado con su trabajo menor es el riesgo de que renuncie a la empresa, por cada punto mayor en la escala el riesgo de renunciar disminuye en 21%

**proyectos** El P-Valor es menor que $\alpha = 0.05$, entonces Se Rechaza $H_0$. Mientras más proyectos asignados tenga el empleado menor es el riesgo de que renuncie a la empresa, por cada trabajo que tenga asignado el riesgo de renunciar disminuye un 25%

**promedio_horas_mes** El P-Valor es menor que $\alpha = 0.05$, entonces Se Rechaza $H_0$. Mientras más horas mensuales trabaja el empleado mayor es el riesgo de que renuncie a la empresa, por cada hora promedio trabajada al mes el riesgo de renunciar aumenta un 0.19%

**accidente** El P-Valor es menor que $\alpha = 0.05$, entonces Se Rechaza $H_0$. Los empleados que han sufrido un accidente dentro de la empresa tienen menor riesgo de renunciar, un 71% menos, que los empleados que no han sufrido accientes

**ascenso_5years** El P-Valor es menor que $\alpha = 0.05$, entonces Se Rechaza $H_0$. Los empleados que han recibido un asecenso en la empresa en los ultimos 5 años tienen menor riesgo de renunciar, un 75% menos, que los empleados que no han recibido ascenso en los ultimos 5 años

**departamento_Investigación** El P-Valor es menor que $\alpha = 0.05$, entonces Se Rechaza $H_0$. Los empleados que pertenecen al departamento de investigación tienen menor riesgo de renunciar, un 30% menos, que los empleados que pertenecen al departamento de desarrollo de producto

**salario_alto** El P-Valor es menor que $\alpha = 0.05$, entonces Se Rechaza $H_0$. Los empleados que tienen un salario alto tienen menor riesgo de renunciar, un 79% menos, que los empleados que tienen un salario bajo


**salario_medio** El P-Valor es menor que $\alpha = 0.05$, entonces Se Rechaza $H_0$. Los empleados que tienen un salario medio tienen menor riesgo de renunciar, un 32% menos, que los empleados que tienen un salario bajo


In [101]:
cph.params_

covariate
satisfaccion                    -0.235066
evaluacion                      -0.013400
proyectos                       -0.282324
promedio_horas_mes               0.001940
accidente                       -1.244632
ascenso_5years                  -1.371784
departamento_Investigación      -0.351695
departamento_administración     -0.126301
departamento_contabilidad       -0.092001
departamento_informática        -0.051288
departamento_mantenimiento       0.087555
departamento_mercadotecnia      -0.006117
departamento_recursos humanos    0.098779
departamento_soporte             0.105733
departamento_ventas              0.006359
salario_alto                    -1.537363
salario_medio                   -0.390725
Name: coef, dtype: float64

In [102]:
cph.hazard_ratios_

covariate
satisfaccion                     0.790519
evaluacion                       0.986689
proyectos                        0.754029
promedio_horas_mes               1.001942
accidente                        0.288047
ascenso_5years                   0.253654
departamento_Investigación       0.703495
departamento_administración      0.881349
departamento_contabilidad        0.912104
departamento_informática         0.950005
departamento_mantenimiento       1.091502
departamento_mercadotecnia       0.993902
departamento_recursos humanos    1.103823
departamento_soporte             1.111525
departamento_ventas              1.006379
salario_alto                     0.214947
salario_medio                    0.676566
Name: exp(coef), dtype: float64

### 5. Con base en la interpretación de los coeficientes, ¿qué recomendación podría hacerle a la empresa para que disminuya la cantidad de renuncias de los empleados?

Los empleados con salarios altos y ascensos recientes presentan mucho menos riesgo de renunciar, yo recomendaría a la empresa considerar dar ascensos minimo cada 5 años a los empleados así como sueldos más competitivos

De igual forma dar más proyectos a los trabajadores y asegurarse que esten satisfechos con su trabajo también podría disminuir las renuncias

### 6. Pruebe si el modelo de Cox cumple el supuesto de riesgos proporcionales, de no hacerlo estime el modelo AFT adecuado

$H_0:$ viola el supuesto

$H_a:$ no se viola el supuesto

In [103]:
cph.check_assumptions(desercion_laboral, p_value_threshold=0.05)


The ``p_value_threshold`` is set at 0.05. Even under the null hypothesis of no violations, some
covariates will be below the threshold by chance. This is compounded when there are many covariates.
Similarly, when there are lots of observations, even minor deviances from the proportional hazard
assumption will be flagged.

With that in mind, it's best to use a combination of statistical tests and visual tests to determine
the most serious violations. Produce visual plots using ``check_assumptions(..., show_plots=True)``
and looking for non-constant lines. See link [A] below for a full example.



<lifelines.StatisticalResult: proportional_hazard_test>
 null_distribution = chi squared
degrees_of_freedom = 1
             model = <lifelines.CoxPHFitter: fitted with 11991 total observations, 10000 right-censored observations>
         test_name = proportional_hazard_test

---
                                    test_statistic      p  -log2(p)
accidente                     km              0.06   0.80      0.32
                              rank            1.33   0.25      2.00
ascenso_5years                km              3.93   0.05      4.40
                              rank            7.11   0.01      7.03
departamento_Investigación    km              0.83   0.36      1.46
                              rank            0.72   0.39      1.34
departamento_administración   km              2.54   0.11      3.17
                              rank            3.18   0.07      3.75
departamento_contabilidad     km              0.21   0.64      0.63
                              rank            0.17   0.68      0.55
departamento_informática      km              0.04   0.83      0.26
                              rank            0.01   0.92      0.12
departamento_mantenimiento    km              0.13   0.72      0.47
                              rank            0.38   0.54      0.90
departamento_mercadotecnia    km              0.58   0.45      1.17
                              rank            3.12   0.08      3.69
departamento_recursos humanos km              0.00   0.94      0.08
                              rank            1.09   0.30      1.76
departamento_soporte          km              1.16   0.28      1.83
                              rank            0.00   1.00      0.00
departamento_ventas           km              0.00   0.99      0.01
                              rank            1.62   0.20      2.30
evaluacion                    km            456.70 <0.005    334.19
                              rank          433.86 <0.005    317.68
promedio_horas_mes            km            262.79 <0.005    193.91
                              rank          303.15 <0.005    223.13
proyectos                     km            564.24 <0.005    411.91
                              rank          685.75 <0.005    499.70
salario_alto                  km              0.36   0.55      0.87
                              rank            0.00   0.96      0.06
salario_medio                 km              0.42   0.52      0.95
                              rank            0.76   0.38      1.38
satisfaccion                  km           1217.52 <0.005    883.70
                              rank          825.64 <0.005    600.75



1. Variable 'satisfaccion' failed the non-proportional test: p-value is <5e-05.

   Advice 1: the functional form of the variable 'satisfaccion' might be incorrect. That is, there
may be non-linear terms missing. The proportional hazard test used is very sensitive to incorrect
functional forms. See documentation in link [D] below on how to specify a functional form.

   Advice 2: try binning the variable 'satisfaccion' using pd.cut, and then specify it in
`strata=['satisfaccion', ...]` in the call in `.fit`. See documentation in link [B] below.

   Advice 3: try adding an interaction term with your time variable. See documentation in link [C]
below.


2. Variable 'evaluacion' failed the non-proportional test: p-value is <5e-05.

   Advice 1: the functional form of the variable 'evaluacion' might be incorrect. That is, there may
be non-linear terms missing. The proportional hazard test used is very sensitive to incorrect
functional forms. See documentation in link [D] below on how to 

[]

Las variables evaluación y satisfacción violan el supuesto de riesgos proporcionales. Hay que estimar otro modelo AFT

In [112]:
mexpo = ExponentialFitter().fit(desercion_laboral['semestre'], desercion_laboral['renunciar'])
mweibull = WeibullFitter().fit(desercion_laboral['semestre'], desercion_laboral['renunciar'])
mlogn = LogNormalFitter().fit(desercion_laboral['semestre'], desercion_laboral['renunciar'])


In [113]:
mexpo.AIC_.round(2), mweibull.AIC_.round(2),mlogn.AIC_.round(2)

(15965.46, 12511.51, 11587.92)

Como el AIC menor es el del modelo LogNorm se estima el modelo AFT Log Normal

In [114]:
from lifelines import LogNormalAFTFitter

In [115]:
LogNormalAFT = LogNormalAFTFitter().fit(desercion_laboral, 'semestre', 'renunciar')
LogNormalAFT.print_summary()

<lifelines.LogNormalAFTFitter: fitted with 11991 total observations, 10000 right-censored observations>
             duration col = 'semestre'
                event col = 'renunciar'
   number of observations = 11991
number of events observed = 1991
           log-likelihood = -4899.95
         time fit was run = 2023-09-26 03:57:27 UTC

---
                                       coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
param  covariate                                                                                                                             
mu_    accidente                       0.26       1.30       0.02             0.22             0.30                 1.25                 1.35
       ascenso_5years                  0.36       1.44       0.06             0.24             0.49                 1.27                 1.63
       departamento_Investigación      0.05       1.05       0.03            -0.02             0.12                 0.98                 1.12
       departamento_administración     0.10       1.10       0.04             0.02             0.18                 1.02                 1.19
       departamento_contabilidad       0.00       1.00       0.03            -0.06             0.07                 0.94                 1.07
       departamento_informática        0.00       1.00       0.03            -0.06             0.06                 0.95                 1.06
       departamento_mantenimiento     -0.03       0.97       0.03            -0.08             0.02                 0.92                 1.02
       departamento_mercadotecnia      0.01       1.01       0.03            -0.05             0.08                 0.95                 1.08
       departamento_recursos humanos  -0.04       0.96       0.03            -0.10             0.02                 0.90                 1.03
       departamento_soporte           -0.03       0.97       0.03            -0.09             0.02                 0.92                 1.02
       departamento_ventas            -0.01       0.99       0.03            -0.06             0.04                 0.94                 1.04
       evaluacion                      0.01       1.01       0.00            -0.00             0.01                 1.00                 1.01
       promedio_horas_mes             -0.00       1.00       0.00            -0.00            -0.00                 1.00                 1.00
       proyectos                       0.09       1.10       0.01             0.08             0.10                 1.09                 1.11
       salario_alto                    0.37       1.45       0.03             0.31             0.43                 1.37                 1.54
       salario_medio                   0.08       1.09       0.01             0.06             0.11                 1.06                 1.11
       satisfaccion                    0.07       1.07       0.00             0.06             0.07                 1.06                 1.07
       Intercept                       0.97       2.64       0.03             0.90             1.03                 2.47                 2.81
sigma_ Intercept                      -1.11       0.33       0.02            -1.14            -1.08                 0.32                 0.34

                                       cmp to      z      p   -log2(p)
param  covariate                                                      
mu_    accidente                         0.00  12.95 <0.005     124.97
       ascenso_5years                    0.00   5.74 <0.005      26.69
       departamento_Investigación        0.00   1.46   0.14       2.80
       departamento_administración       0.00   2.57   0.01       6.64
       departamento_contabilidad         0.00   0.11   0.91       0.13
       departamento_informática          0.00   0.07   0.94       0.08
       departamento_mantenimiento        0.00  -1.11   0.27       1.91
       departamento_mercadotecnia 

### 7. Interprete los coeficientes del modelo AFT

**evaluacion, departamento_Investigación, departamento_contabilidad, departamento_informática, departamento_mantenimiento, departamento_mercadotecnia, departamento_recursos humanos, departamento_soporte, departamento_ventas** Su P-valor es mayor que el $\alpha = 0.05$, por lo que NO se rechaza $H_0$, lo que quiere decir que estas variables no tiene efecto en el tiempo promedio que un empleado tarda en renunciar a la empresa

**accidente** El P-Valor es menor que $\alpha = 0.05$, entonces Se Rechaza $H_0$. Haber sufrido un accidente dentro de la empresa aumenta el tiempo promedio (media) que un empleado tarda en renunciar en un 30% en comparación con los empleados que no sufrieron un accidente

**ascenso_5years** El P-Valor es menor que $\alpha = 0.05$, entonces Se Rechaza $H_0$. Haber tenido unascenso en los ultimos 5 años aumenta el tiempo promedio (media) que un empleado tarda en renunciar en un 44% más en comparación con los empleados que no han recibido ascenso en los ultimos 5 años

**departamento_administración** El P-Valor es menor que $\alpha = 0.05$, entonces Se Rechaza $H_0$. Los empleados que trabajan en el departamento de administración tardan 10% más tiempo promedio en renunciar a la empresa que los que trabajan en desarrollo de producto

**promedio_horas_mes** El P-Valor es menor que $\alpha = 0.05$, entonces Se Rechaza $H_0$. Por cada hora promedio al mes que trabaje el empleado el tiempo promedio que pasa antes de que renuncie disminuye cerca del 0%

**proyectos** El P-Valor es menor que $\alpha = 0.05$, entonces Se Rechaza $H_0$. Los empleados que tienen más proyectos tardan 10% más tiempo promedio en renunciar a la empresa por cada proyecto más que tengan

**salario_alto** El P-Valor es menor que $\alpha = 0.05$, entonces Se Rechaza $H_0$. Los empleados que tienen un salario alto tardan 45% más tiempo promedio en renunciar a la empresa que los que tienen salario bajo

**salario_medio** El P-Valor es menor que $\alpha = 0.05$, entonces Se Rechaza $H_0$. Los empleados que tienen un salario medio tardan 9% más tiempo promedio en renunciar a la empresa que los que tienen salario bajo

**satisfaccion** El P-Valor es menor que $\alpha = 0.05$, entonces Se Rechaza $H_0$. Mientras más satisfecho se encuentre el empleado con su trabajo más tiempo proemedio pasa antes de que renuncie a la empresa, por cada punto mayor en la escala el la media del tiempo antes de renunciar aumenta en 7%